In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
cust_data = pd.read_csv('data_transform.csv')
cust_data = cust_data[['TARGET',
'AGE',
'ARPU',
'AUTR_FAIL_MCNT',
'AVG_CALL_TIME',
'CPT_LNIF_AMT',
'CRDT_CARD',
'CRDT_CARD_CNT',
'CRDT_OCCR_MDIF',
'CRMM_OVDU_AMT',
'CTCD_OCCR_MDIF',
'HIGH_AMT_RATE',
'L_H_RATE',
'LNIF_CNT',
'LOW_AMT_RATE',
'LT1Y_CLOD_RATE',
'LT1Y_CTLT_CNT',
'LT1Y_MXOD_AMT',
'MOBL_PRIN',
'MON_TLFE_AMT',
'NUM_DAY_SUSP',
'OVDU_HIGH_RATE',
'PAYM_METD_G',
'PREM_OVDU_RATE',
'SPTCT_OCCR_MDIF',
'TEL_CNTT_QTR',
'TEL_OVDU_RATE',
'TOT_LNIF_AMT',
'TOT_LOAN_CNT',
'CPT_LNIF_BIG',
'CPT_LNIF_RATIO',
'CRDT_CARD',
'CRDT_GRAD_DIFF',
'DTI',
'FAIL_COUNT',
'HIGH_AMT_RATE',
'HSHD_INFR_INCM',
'LNIF_CNT',
'LOW_AMT_RATE'
]]
cust_data.shape

(100233, 39)

In [3]:
cust_data.head()

,TARGET,AGE,ARPU,AUTR_FAIL_MCNT,AVG_CALL_TIME,CPT_LNIF_AMT,CRDT_CARD,CRDT_CARD_CNT,CRDT_OCCR_MDIF,CRMM_OVDU_AMT,...,CPT_LNIF_BIG,CPT_LNIF_RATIO,CRDT_CARD,CRDT_GRAD_DIFF,DTI,FAIL_COUNT,HIGH_AMT_RATE,HSHD_INFR_INCM,LNIF_CNT,LOW_AMT_RATE
0,0,50,30000,10,450,0,26,2,1,0,...,False,0.0,26,0,1.667,10,9002.0,7700,1,0.000000
1,0,50,30000,0,81,0,242,2,0,0,...,False,0.0,242,0,4.364,0,1.0,8100,1,0.000000
2,0,60,30000,0,139,3001,484,4,1,0,...,False,0.2,484,0,0.000,0,9001.0,4900,6,0.200053
3,1,35,30000,0,1118,3001,244,4,1,540000,...,False,0.5,244,0,0.000,0,3001.0,10100,8,0.500083
4,0,45,50000,0,396,0,97,1,1,130000,...,False,0.0,97,0,4.375,0,15002.0,4800,4,0.000000


In [4]:
def model_performance(y_test, y_pred):    
    print('confusion matrix')
    print(metrics.confusion_matrix(y_test, y_pred))
    print('accuracy : {}'.format(metrics.accuracy_score(y_test, y_pred).round(3)))
    print('precision : {}'.format(metrics.precision_score(y_test, y_pred, pos_label=1).round(3)))
    print('recall : {}'.format(metrics.recall_score(y_test, y_pred, pos_label=1).round(3)))
    print('F1 : {}'.format(metrics.f1_score(y_test, y_pred, pos_label=1).round(3)))

## 연체자 data duplicate

In [5]:
cust = cust_data[cust_data['TARGET'] == 0]
cust_overdue = cust_data[cust_data['TARGET'] == 1]     # 연체자

print(cust.shape[0])
print(cust_overdue.shape[0])

95946
4287


In [6]:
variables = cust_data.drop('TARGET', 1).columns
variables

Index(['AGE', 'ARPU', 'AUTR_FAIL_MCNT', 'AVG_CALL_TIME', 'CPT_LNIF_AMT',
       'CRDT_CARD', 'CRDT_CARD_CNT', 'CRDT_OCCR_MDIF', 'CRMM_OVDU_AMT',
       'CTCD_OCCR_MDIF', 'HIGH_AMT_RATE', 'L_H_RATE', 'LNIF_CNT',
       'LOW_AMT_RATE', 'LT1Y_CLOD_RATE', 'LT1Y_CTLT_CNT', 'LT1Y_MXOD_AMT',
       'MOBL_PRIN', 'MON_TLFE_AMT', 'NUM_DAY_SUSP', 'OVDU_HIGH_RATE',
       'PAYM_METD_G', 'PREM_OVDU_RATE', 'SPTCT_OCCR_MDIF', 'TEL_CNTT_QTR',
       'TEL_OVDU_RATE', 'TOT_LNIF_AMT', 'TOT_LOAN_CNT', 'CPT_LNIF_BIG',
       'CPT_LNIF_RATIO', 'CRDT_CARD', 'CRDT_GRAD_DIFF', 'DTI', 'FAIL_COUNT',
       'HIGH_AMT_RATE', 'HSHD_INFR_INCM', 'LNIF_CNT', 'LOW_AMT_RATE'],
      dtype='object')

### ML modeling

In [7]:
# train / test set 분리
x = cust_data.drop('TARGET', axis=1)
y = cust_data['TARGET']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print('연체자 수 train : {} / {}'.format(sum(y_train == 1), y_train.shape[0]))
print('연체자 수 test  : {} / {}'.format(sum(y_test  == 1), y_test.shape[0]))

연체자 수 train : 3439 / 80186
연체자 수 test  : 848 / 20047


In [8]:
# train set 중에서 연체자만 추출
x_overdue = x_train[y_train == 1]
y_overdue = y_train[y_train == 1]
print(x_overdue.shape)
print(y_overdue.shape)

(3439, 38)
(3439,)


In [9]:
max_epoch = 5

for i in range(max_epoch+1):
    x2_train, x2_test, y2_train, y2_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0)
    
    print('========================================= ', i)
    print('test {} : test overdue {}'.format(y2_test.shape[0], sum(y2_test == 1)))
    print('')
    
#     clf = DecisionTreeClassifier()
    clf = GradientBoostingClassifier(n_estimators=500, random_state=0)
    
    clf.fit(x2_train, y2_train)
    
#     y_pred = clf.predict(x2_test)
#     model_performance(y2_test, y_pred)
    
#     varDic = {'var':variables, 'importance':clf.feature_importances_}
#     importance = pd.DataFrame(varDic)
#     print(importance.sort_values(by='importance', ascending=False).head(10))

    x_train = pd.concat([x_train, x_overdue])
    y_train = pd.concat([y_train, y_overdue])

#     if i == max_epoch:
    print('------------------------')
    y_pred_0 = clf.predict(x_test)
    model_performance(y_test, y_pred_0)
    print('------------------------')

=========================================  0
test 16038 : test overdue 720

------------------------
confusion matrix
[[19061   138]
 [  664   184]]
accuracy : 0.96
precision : 0.571
recall : 0.217
F1 : 0.315
------------------------
=========================================  1
test 16725 : test overdue 1415

------------------------
confusion matrix
[[18895   304]
 [  560   288]]
accuracy : 0.957
precision : 0.486
recall : 0.34
F1 : 0.4
------------------------
=========================================  2
test 17413 : test overdue 2095

------------------------
confusion matrix
[[18719   480]
 [  486   362]]
accuracy : 0.952
precision : 0.43
recall : 0.427
F1 : 0.428
------------------------
=========================================  3
test 18101 : test overdue 2740

------------------------
confusion matrix
[[18538   661]
 [  437   411]]
accuracy : 0.945
precision : 0.383
recall : 0.485
F1 : 0.428
------------------------
=========================================  4
test 18789 : test